In [129]:
import pandas as pd

## Загружаем исходные данные

In [130]:
visits = pd.read_csv('/content/drive/MyDrive/WS_FINAL_2023/data/visits_info_short.csv')
orders = pd.read_csv('/content/drive/MyDrive/WS_FINAL_2023/data/orders_info_short.csv')
costs = pd.read_csv('/content/drive/MyDrive/WS_FINAL_2023/data/costs_info_short.csv')

# Предобработка

In [131]:
# заменить пробелы в названиях столбцов на нижнее подчеркивание
visits.columns = [x.lower().replace(' ', '_') for x in visits.columns]
orders.columns = [x.lower().replace(' ', '_') for x in orders.columns]
costs.columns = costs.columns.str.lower()

In [132]:
# заменить пробелы в названиях столбцов на нижнее подчеркивание
visits.columns = [x.lower().replace(' ', '_') for x in visits.columns]
orders.columns = [x.lower().replace(' ', '_') for x in orders.columns]
costs.columns = costs.columns.str.lower()
costs['dt'] = pd.to_datetime(costs['dt'])# меняем тип даты

In [133]:
visits

,user_id,region,device,channel,session_start,session_end
0,981449118918,United States,iPhone,organic,2019-05-01 02:36:01,2019-05-01 02:45:01
1,278965908054,United States,iPhone,organic,2019-05-01 04:46:31,2019-05-01 04:47:35
2,590706206550,United States,Mac,organic,2019-05-01 14:09:25,2019-05-01 15:32:08
3,326433527971,United States,Android,TipTop,2019-05-01 00:29:59,2019-05-01 00:54:25
4,349773784594,United States,Mac,organic,2019-05-01 03:33:35,2019-05-01 03:57:40
...,...,...,...,...,...,...
309896,329994900775,UK,PC,LeapBob,2019-10-31 13:28:12,2019-10-31 14:39:29
309897,334903592310,France,PC,lambdaMediaAds,2019-10-31 22:14:52,2019-10-31 22:39:36
309898,540102010126,Germany,PC,organic,2019-10-31 01:40:48,2019-10-31 01:41:31
309899,308736936846,Germany,Mac,organic,2019-10-31 07:37:34,2019-10-31 07:37:55


In [134]:
# перевод значений столбцов из строки в дату
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_end'])
orders['event_dt'] = pd.to_datetime(orders['event_dt'])
costs['dt'] = pd.to_datetime(costs['dt']).dt.date

In [135]:
def get_profiles(sessions, orders, costs):

    # находим параметры первых посещений
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()
    )

    # для когортного анализа определяем дату первого посещения
    # и первый день месяца, в который это посещение произошло
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].dt.month

    # добавляем признак платящих пользователей
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())

    # считаем количество уникальных пользователей
    # с одинаковыми источником и датой привлечения
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()
    )

    # объединяем траты на рекламу и число привлечённых пользователей
    costs = costs.merge(new_users, on=['dt', 'channel'], how='left')

    # делим рекламные расходы на число привлечённых пользователей
    costs['acquisition_cost'] = costs['costs'] / costs['unique_users']

    # добавляем стоимость привлечения в профили
    profiles = profiles.merge(
        costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # стоимость привлечения органических пользователей равна нулю
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)

    return profiles

In [136]:
profiles = get_profiles(visits, orders, costs)
profiles.head(5)

,user_id,first_ts,channel,device,region,dt,month,payer,acquisition_cost
0,599326,2019-05-07 20:58:57,FaceBoom,Mac,United States,2019-05-07,5,True,1.088172
1,4919697,2019-07-09 12:46:07,FaceBoom,iPhone,United States,2019-07-09,7,False,1.107237
2,6085896,2019-10-01 09:58:33,organic,iPhone,France,2019-10-01,10,False,0.000000
3,22593348,2019-08-22 21:35:48,AdNonSense,PC,Germany,2019-08-22,8,False,0.988235
4,31989216,2019-10-02 00:07:44,YRabbit,iPhone,United States,2019-10-02,10,False,0.230769


In [137]:
# функция для расчёта конверсии

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # исключаем пользователей, не «доживших» до горизонта анализа
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # определяем дату и время первой покупки для каждого пользователя
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # собираем «сырые» данные для расчёта удержания
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )


    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days


    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # функция для группировки таблицы по желаемым признакам
    def group_by_dimensions(df, dims, horizon_days):
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        result = result.fillna(0).cumsum(axis = 1)
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # делим каждую «ячейку» в строке на размер когорты
        # и получаем conversion rate
        result = result.div(result['cohort_size'], axis=0)
        result = result[['cohort_size'] + list(range(horizon_days))]
        result['cohort_size'] = cohort_sizes
        return result

    # получаем таблицу конверсии
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)


    if 'cohort' in dimensions:
        dimensions = []

    # получаем таблицу динамики конверсии
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # возвращаем обе таблицы и сырые данные
    return result_raw, result_grouped, result_in_time

In [138]:
import datetime
from datetime import date, datetime, timedelta
#profiles = profiles.query('channel != "organic"') #исключаем из таблицы органических пользователей.
observation_date = datetime(2019, 11, 1).date()  # задаём момент анализа
horizon_days = 14  #задаём горизонт анализа

In [139]:
conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days
)

In [140]:
conversion_history

,cohort_size,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0
dt,,,,,,,,,,,,,,,
2019-05-01,909,0.031903,0.036304,0.038504,0.038504,0.040704,0.040704,0.040704,0.041804,0.042904,0.044004,0.044004,0.044004,0.044004,0.045105
2019-05-02,758,0.029024,0.035620,0.036939,0.036939,0.036939,0.036939,0.036939,0.038259,0.038259,0.038259,0.039578,0.039578,0.039578,0.039578
2019-05-03,849,0.031802,0.037691,0.043581,0.044759,0.047114,0.047114,0.047114,0.047114,0.047114,0.047114,0.047114,0.047114,0.047114,0.048292
2019-05-04,1015,0.035468,0.039409,0.040394,0.042365,0.047291,0.047291,0.048276,0.050246,0.051232,0.052217,0.052217,0.052217,0.052217,0.052217
2019-05-05,970,0.023711,0.026804,0.028866,0.030928,0.032990,0.032990,0.032990,0.032990,0.032990,0.032990,0.032990,0.032990,0.032990,0.032990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-10-15,674,0.047478,0.048961,0.051929,0.053412,0.057864,0.059347,0.060831,0.060831,0.060831,0.062315,0.062315,0.062315,0.062315,0.062315
2019-10-16,656,0.033537,0.036585,0.042683,0.042683,0.042683,0.044207,0.044207,0.045732,0.048780,0.048780,0.048780,0.048780,0.048780,0.050305
2019-10-17,703,0.038407,0.042674,0.048364,0.049787,0.051209,0.051209,0.052632,0.052632,0.054054,0.054054,0.054054,0.054054,0.054054,0.054054


In [141]:
profiles.to_csv("profiles.csv")

# Кодируем текстовые данные

In [142]:
df = profiles.drop(["first_ts", "dt","user_id",'acquisition_cost'], axis=1)

In [143]:
from sklearn.preprocessing import LabelEncoder
# Выбираме колонки для кодирования
columns_to_encode = ['channel', 'device', 'region', 'payer']

encoders = {}
# Перебираем циклом наши колоки и кодируем их
for column in columns_to_encode:
  # Инициализируем LabelEncoder
    label_encoder = LabelEncoder()
    df[column] = label_encoder.fit_transform(df[column])
    encoders[column] = label_encoder

# Смотрим)
df

,channel,device,region,month,payer
0,1,1,3,5,1
1,1,3,3,7,0
2,10,3,0,10,0
3,0,2,1,8,0
4,8,3,3,10,0
...,...,...,...,...,...
150003,6,3,3,9,0
150004,10,2,2,10,0
150005,6,3,3,7,0
150006,3,2,3,9,0


In [144]:
encoders['channel'].transform(encoders['channel'].classes_)



# encoded_df['Shape'] = encoder.fit_transform(df['Shape'])
# encoding_mapping['Shape'] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [145]:
encoders['channel'].classes_


array(['AdNonSense', 'FaceBoom', 'LeapBob', 'MediaTornado',
       'OppleCreativeMedia', 'RocketSuperAds', 'TipTop', 'WahooNetBanner',
       'YRabbit', 'lambdaMediaAds', 'organic'], dtype=object)

In [146]:
# Экпортируем классы для дальнейшего декондинга
import joblib
joblib.dump(encoders, 'label_encoder.pkl')

#Если надо декодировать
#loaded_label_encoder = joblib.load('label_encoder.pkl')

# new_df = pd.DataFrame(new_data)

#for column in columns_to_encode:
    #new_df[column] = loaded_label_encoder.inverse_transform(new_df[column])


['label_encoder.pkl']

Разобьем данные на кластеры

In [ ]:
from sklearn.metrics import silhouette_score
# Метод "силуэт" для определения оптимального числа кластеров
silhouette_scores = []
for k in range(2, 9):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df)
    labels = kmeans.labels_
    score = silhouette_score(df, labels)
    silhouette_scores.append(score)

# График "силуэта"
plt.plot(range(2, 9), silhouette_scores, marker='o')
plt.xlabel('Количество кластеров')
plt.ylabel('Оценка силуэта')
plt.title('Метод "силуэта" для определения оптимального числа кластеров')
plt.show()

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from sklearn.cluster import KMeans

# Создание объекта KMeans с указанием числа кластеров (например, 3)
kmeans = KMeans(n_clusters=2)

# Применение метода кластеризации к данным
kmeans_clusters = kmeans.fit_predict(df)
dfdf['kmeans_cluster'] = kmeans_clusters

In [ ]:
common_df = df.copy()
common_df["acquisition_cost"] = profiles["acquisition_cost"]
common_df["channel"] = profiles["channel"]
# Визуализация кластеров
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(8, 6))
plt.xticks(rotation=90)
sns.scatterplot(x='channel', y='acquisition_cost', hue='kmeans_cluster', data=common_df, palette='viridis', s=100, alpha=0.8)
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], c='red', marker='X', s=200, label='Centroids')
plt.title('Визуализация кластеров')
plt.legend()
plt.show()

Научим модель предсказывать сегмент

Разделим данные на выборки

In [ ]:
from sklearn.model_selection import train_test_split
# разделим данные на обучающие (X) и целевую  (y)
X = df.drop('kmeans_cluster', axis=1)
y = df['kmeans_cluster']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Классификатор


In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train, y_train)


y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", confusion_mat)
print("Classification Report:\n", classification_rep)



In [ ]:
import joblib
joblib.dump(classifier, 'segment_classificator.pkl')